# 射影方策勾配法の収束レート ($1/k$)

参考：
* [On the Convergence Rates of Policy Gradient Methods](https://www.jmlr.org/papers/volume23/22-0056/22-0056.pdf)

Direct parameterizationの元で，次の射影方策勾配法を考えましょう（今回はコストの最小化を考えます）．
$$
\pi^{(k+1)}=\operatorname{proj}_{\Pi}\left(\pi^{(k)}-\eta_k \nabla V_\mu\left(\pi^{(k)}\right)\right)
$$

[RL_PolicyGrad_convergence_rate.ipynb](RL_PolicyGrad_convergence_rate.ipynb)では$1/\sqrt{k}$のレートを示しましたが，実は$1/k$でいけます．証明しましょう．
前提として，Gradient dominanceは変わらず使用します:

$$
V_\rho(\pi)-V_\rho^{\star} \leq \frac{1}{1-\gamma}\left\|\frac{d_\rho\left(\pi^{\star}\right)}{\mu}\right\|_{\infty} \max _{\pi^{\prime} \in \Pi}\left\langle\nabla V_\mu(\pi), \pi-\pi^{\prime}\right\rangle
$$



## 一般の近接勾配法についての収束を示す

射影方策勾配法は近接勾配法の特殊ケースなので，近接勾配法の結果を示せばレートが出せます．
そこで，次のComposite optimizationを考えましょう：
$$
\underset{x \in \mathbf{R}^n}{\operatorname{minimize}} F(x):=f(x)+\Psi(x),
$$
$f$と$\Psi$に仮定をおきます：
* $f$はsmooth：$\|\nabla f(x)-\nabla f(y)\|_2 \leq L\|x-y\|_2, \quad \forall x, y \in \operatorname{dom} \Psi$
* $\Psi$はconvexかつ下半連続

### 準備

$x \leftarrow \pi, f \leftarrow V_\rho$，$\Psi$を$\Pi$についての指示関数とすれば，MDPでの最適化はcomposite optimizationの特殊ケースになります．

そして，射影方策勾配法は次のように近接勾配法で表現できます：
$$
\begin{aligned}
x^{(k+1)} & =\underset{x}{\arg \min }\left\{\left\langle\nabla f\left(x^{(k)}\right), x-x^{(k)}\right\rangle+\frac{1}{2 \eta_k}\left\|x-x^{(k)}\right\|^2+\Psi(x)\right\} \\
& =\operatorname{prox}_{\eta_k \Psi}\left(x^{(k)}-\eta_k \nabla f\left(x^{(k)}\right)\right)
\end{aligned}
$$
ここで，
$$
\operatorname{prox}_\phi(x)=\underset{y}{\arg \min }\left\{\phi(y)+\frac{1}{2}\|y-x\|_2^2\right\}
$$
です．
表記として，
$$
T_L(x):=\operatorname{prox}_{\frac{1}{L} \Psi}\left(x-\frac{1}{L} \nabla f(x)\right)
$$
および
$$
G_L(x):=L\left(x-T_L(x)\right)
$$
を導入します．

### 収束解析（Agarwalのやりかた）

[RL_PolicyGrad_convergence_rate.ipynb](RL_PolicyGrad_convergence_rate.ipynb)では，次のDescent lemmaを使って収束を証明しました:

$$
F\left(x^{(k)}\right)-F\left(x^{(k+1)}\right) \geq \frac{1}{2 L}\left\|G_L\left(x^{(k)}\right)\right\|_2^2
$$

これは[Gradient methods for minimizing composite objective function](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=e61f2788481ab6049436b6784d619439ced56575)のTheorem 1にあります．

Telescopingをして，
$$
F\left(x^{(0)}\right)-F\left(x^{(K+1)}\right) \geq \frac{1}{2 L} \sum_{k=0}^K\left\|G_L\left(x^{(k)}\right)\right\|_2^2
$$
が得られます．最後に，
$$
\min _{0 \leq k \leq K}\left\|G_L\left(x^{(k)}\right)\right\|_2 \leq \sqrt{\frac{2 L\left(F\left(x^{(0)}\right)-F^{\star}\right)}{K+1}} .
$$

で，停留点に$1/\sqrt{K}$のレートで収束することがわかります．

最後に，[Gradient methods for minimizing composite objective function](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=e61f2788481ab6049436b6784d619439ced56575)のTheorem 1を使って，
$$
\left\langle\nabla V_\mu\left(T_L(\pi)\right), T_L(\pi)-\pi^{\prime}\right\rangle \leq 2\left\|G_L(\pi)\right\|_2 \cdot\underbrace{\left\|T_L(\pi)-\pi^{\prime}\right\|_2}_{\leq \sqrt{2|\mathcal{S}|}}, \quad \forall \pi, \pi^{\prime} \in \Pi
$$
が言えるので，Gradient dominanceと合わせて，

$$
\begin{aligned}
V_\rho(\pi^{(k+1)})-V_\rho^{\star} 
&\leq \frac{1}{1-\gamma}\left\|\frac{d_\rho\left(\pi^{\star}\right)}{\mu}\right\|_{\infty} \max _{\pi^{\prime} \in \Pi}\left\langle\nabla V_\mu(\pi^{(k+1)}), \pi^{(k+1)}-\pi^{\prime}\right\rangle\\
&\leq \square \left\|G_L(\pi^{(k)})\right\|_2
\end{aligned}
$$

で，証明終わりです．


## 収束解析（強め）

上のやり方でもいいんですが，実はGradient dominanceを収束レートに適用すれば，もっといいバウンドが出せます．

そのために，次のWeak Gradient-Mapping Dominationを定義しましょう．

---

**Weak Gradient-Mapping Domination**

$F = f + \Psi$は，次を満たす時に指数$\alpha \in (0.5, 1]$のWeak Gradient-Mapping Dominationを満たすといいます：
次を満たす$\omega > 0$が存在する：
$$
\left\|G_L(x)\right\|_2 \geq \sqrt{2 \omega}\left(F\left(T_L(x)\right)-F^{\star}\right)^\alpha, \quad \forall x \in \operatorname{dom} \Psi
$$

---

これは微分可能＆Globalな[KL条件](OPT_Grad_KL_condition.ipynb)とほとんど同じです．
KL条件では，
1. $\alpha = 0$ならば，$\{x^k\}$は有限回で収束する
2. $\alpha \in (0, 1/2]$ならば，$\{x^k\}$は線形に収束する
2. $\alpha \in (1/2, 1)$ならば，$\{x^k\}$は劣線形に収束する

が一般に言えます．しかし，今回は$\alpha = 1$のときが知りたいので，KL条件とは少し違います．
この条件が成立する時の収束の解析をしましょう．

### 近接勾配の収束解析

学習率$\eta_k = 1 / L$とします．このとき，Descent lemmaとWeak Gradient-Mapping Dominationを使うと，

$$
F\left(x^{(k)}\right)-F\left(x^{(k+1)}\right) \geq \frac{1}{2 L}\left\|G_L\left(x^{(k)}\right)\right\|_2^2 \geq \frac{\omega}{L}\left(F\left(x^{(k+1)}\right)-F^{\star}\right)^{2 \alpha}
$$

が成り立ちます．最適解との誤差を$\delta_k=F\left(x^{(k)}\right)-F^{\star} \geq 0$とすると，上の式から
$$
\delta_k-\delta_{k+1} \geq \frac{\omega}{L} \delta_{k+1}^{2 \alpha}
$$
です．ここで，$\alpha \in (1/2, 1]$なので，$2\alpha - 1 \in (0, 1]$です．

一旦次のTelescopingを考えましょう：

$$
\frac{1}{\delta_k^{2 \alpha-1}}-\frac{1}{\delta_0^{2 \alpha-1}}=\sum_{i=0}^{k-1}\left(\frac{1}{\delta_{i+1}^{2 \alpha-1}}-\frac{1}{\delta_i^{2 \alpha-1}}\right)=\sum_{i=0}^{k-1} \frac{2 \alpha-1}{\delta_{i+1}^{2 \alpha-1}} \frac{1}{2 \alpha-1}\left(1-\left(\frac{\delta_{i+1}}{\delta_i}\right)^{2 \alpha-1}\right)
$$

が成り立ちます．
ここで，descent lemmaから，$\delta_{i+1} / \delta_i \leq 1$が言えます．
また，任意の$\beta > 0$と$\rho \in [0, 1]$について，
$$
\frac{1}{\beta}\left(1-\rho^\beta\right) \geq \rho^\beta(1-\rho)
$$
が成り立つので（証明は略），$\beta = 2\alpha - 1$と$\rho=\delta_{i+1} / \delta_i$について，
$$
\begin{aligned}
\frac{1}{\delta_k^{2 \alpha-1}}-\frac{1}{\delta_0^{2 \alpha-1}} 
&\geq \sum_{i=0}^{k-1} \frac{2 \alpha-1}{\delta_{t+1}^{2 \alpha-1}}\left(\frac{\delta_{i+1}}{\delta_i}\right)^{2 \alpha-1}\left(1-\frac{\delta_{i+1}}{\delta_i}\right)
=(2 \alpha-1) \sum_{i=0}^{k-1} \frac{\delta_i-\delta_{i+1}}{\delta_i^{2 \alpha}}\\
&\geq(2 \alpha-1) \frac{\omega}{L}
\sum_{i=0}^{k-1} \left(\frac{\delta_{i+1}}{\delta_i}\right)^{2 \alpha}
\end{aligned}
$$

が言えます．最後の不等式では上で示したdescent lemmaの応用を使ってます．

ここで，$n(k, r)$を，$k$回のイテレーション内で，$\frac{\delta_{i+1}}{\delta_i} \geq r$が成立した回数の下界とします．
つまり，$n(k, r) \geq ck$なら，$\delta_{i+1} / \delta_i \geq r$が最低でも$\lceil c k\rceil$回起きてます．
これを使って，場合分けしてレートを導出しましょう．

**1. $n(k, r) \geq ck$のとき**

$$
\frac{1}{\delta_k^{2 \alpha-1}}-\frac{1}{\delta_0^{2 \alpha-1}} \geq(2 \alpha-1) \frac{\omega}{L} r^{2 \alpha} c k
$$
とできます．変形して，
$$
\delta_k \leq\left(\frac{1}{1 / \delta_0^{2 \alpha-1}+(2 \alpha-1) \frac{\omega}{L} r^{2 \alpha} c k}\right)^{\frac{1}{2 \alpha-1}} \leq\left(\frac{L}{(2 \alpha-1) \omega r^{2 \alpha} c}\right)^{\frac{1}{2 \alpha-1}}\left(\frac{1}{k}\right)^{\frac{1}{2 \alpha-1}}
$$
が成り立ちます．

**2. $n(k, r) < ck$のとき**

$\delta_{i+1} / \delta_i < r$が最低でも$\lceil(1-c) k\rceil$回成立しています．
よって，
$$
\delta_{i+1} \leq \delta_i \leq \delta_{i-1} \leq \cdots \leq \delta_0
$$
なので，$\delta_k \leq \delta_0 r^{(1-c) k}=\delta_0\left(r^{1-c}\right)^k$です．

場合分けをまとめると，$r, c \in (0, 1)$が任意に選択できるので，

$$
\delta_k \leq \min _{0<r, c<1} \max \left\{\left(\frac{L}{(2 \alpha-1) \omega r^{2 \alpha} c}\right)^{\frac{1}{2 \alpha-1}}\left(\frac{1}{k}\right)^{\frac{1}{2 \alpha-1}},\left(r^{1-c}\right)^k \delta_0\right\}
$$

が成り立ちます．最後に$r=c=1/2$として，

$$
F\left(x^{(k)}\right)-F^{\star} \leq \max \left\{\left(\frac{2^{2 \alpha+1} L}{(2 \alpha-1) \omega}\right)^{\frac{1}{2 \alpha-1}}\left(\frac{1}{k}\right)^{\frac{1}{2 \alpha-1}},\left(\frac{\sqrt{2}}{2}\right)^k\left(F\left(x^{(0)}\right)-F^{\star}\right)\right\} .
$$

が成り立ちます．特に$\alpha=1$のとき，
$$
F\left(x^{(k)}\right)-F^{\star} \leq \max \left\{\frac{8 L}{\omega k},\left(\frac{\sqrt{2}}{2}\right)^k\left(F\left(x^{(0)}\right)-F^{\star}\right)\right\}
$$
です．


### 方策勾配に適用

以上の結果を方策勾配法に適用しましょう．
Agarwalのやりかたでやったように，Gradient dominanceを使えば
$$
\begin{aligned}
V_\rho(\pi^{(k+1)})-V_\rho^{\star} 
&\leq \frac{1}{1-\gamma}\left\|\frac{d_\rho\left(\pi^{\star}\right)}{\mu}\right\|_{\infty} \max _{\pi^{\prime} \in \Pi}\left\langle\nabla V_\mu(\pi^{(k+1)}), \pi^{(k+1)}-\pi^{\prime}\right\rangle
\end{aligned}
$$
を使いたいところですが，左辺と右辺の初期状態分布が同じでないと，Weak Gradient-Mapping Dominationが満たされません．そこでまず両辺を同じにして，weak gradient-mapping dominationの$\alpha=1$を考え，
$$
V_\rho\left(\pi^{(k)}\right)-V_\rho^{\star} \leq \frac{256|\mathcal{S} \| \mathcal{A}|}{k(1-\gamma)^5}\left\|\frac{d_\mu\left(\pi^{\star}\right)}{\mu}\right\|_{\infty}^2
$$
を出します．そして，$\rho$については
$$
\begin{aligned}
V_\rho\left(\pi^{(k)}\right)-V_\rho^{\star} & =\sum_{s \in \mathcal{S}} \rho_s\left(V_s\left(\pi^{(k)}\right)-V_s^{\star}\right) \\
& =\sum_{s \in \mathcal{S}} \frac{\rho_s}{\mu_s} \mu_s\left(V_s\left(\pi^{(k)}\right)-V_s^{\star}\right) \\
& \leq\left\|\frac{\rho}{\mu}\right\|_{\infty}\left(V_\mu\left(\pi^{(k)}\right)-V_\mu^{\star}\right) .
\end{aligned}
$$
とすれば大丈夫です．

**注意**：最初の等式が成り立つのは多分MDPが一様最適な方策を持つからだと思われる．CMDPとかはやっちゃだめそう．